<h1>Считывание данных</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file = "aes_data.xlsx"

data = pd.read_excel(file)

df = data.drop([0,1,2], axis=0)
cols_fillna = df.columns.values
for col in cols_fillna[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    
pump_in_cols = df.columns.values[1:6]
pump_out_cols = df.columns.values[6:11]
pump_mass_cols = df.columns.values[11:16]
pump_temp_col = df.columns.values[16]
pump_amper_cols = df.columns.values[17:22]
pump_revs_cols = df.columns.values[22:27]
pump_volt_cols = df.columns.values[27:31]
pump_turbine_col = df.columns.values[31]
pump_reactor_col = df.columns.values[32]


# df[cols_fillna[-1]][df[cols_fillna[-1]].isnull()]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data.head()


<h1>Первичная фильтрация данных</h1>
<p>убираем отрицательные или слишком большие значения</p>
<p>+ графики массовой подачи для всех насосов</p>


In [ ]:
# df["10LAB13CP002_XQ01"] = df.loc[df["10LAB13CP002_XQ01"]>8, "10LAB13CP002_XQ01"].apply(lambda x: x-x//1)


# df["10LAB13CP002_XQ01"] = df["10LAB13CP002_XQ01"].apply(lambda x: x if x<=8 else x-x//1)   
df[pump_in_cols[2]] = df[pump_in_cols[2]].apply(lambda x: x if x<=8 else x-x//1)   


for col in df.columns.values[1:]:
    df[col] = df[col].apply(lambda x: 0 if x<0 else x)

# pump_columns = ["10LAB11CP002_XQ01","10LAB12CP002_XQ01","10LAB13CP002_XQ01","10LAB14CP002_XQ01","10LAB15CP002_XQ01"]
# pump_columns = ["10LAB11CF001_XQ01","10LAB12CF001_XQ01","10LAB13CF001_XQ01","10LAB14CF001_XQ01","10LAB15CF001_XQ01"]

for col in pump_mass_cols:
    df[col] = df[col].apply(lambda x: x if x<510 else 0)    

    
plt.rcParams["figure.figsize"] = (20,10)

plot = df.plot(use_index=True, y=pump_mass_cols)
plt.title('Показания ПЭНов')
plt.ylabel('Массовая подача, кг/с', fontsize=15)
plt.legend(loc='lower right')
plt.show()


<h1>Расчеты КПД</h1>

In [ ]:
from iapws import IAPWS97, IAPWS95

def eff_func(row, pump_num):
    p1 = df[f'10LAB{pump_num}CP002_XQ01'][row]
    p2 = df[f'10LAB{pump_num}CP003_XQ01'][row]
    G = df[f'10LAB{pump_num}CF001_XQ01'][row]
    t = df["10LAB00FT902_XQ01"][row]
    I = df[f"10LAC{pump_num}CE021_XQ01"][row]
    U = df["10BBA00CE010_XQ01"][row]
    
    if all([(p2-p1)*G,U*I, t, p2, p1]):
        try:
            rho = IAPWS97(P=(p1+p2)/2, T=t+273).rho
        except:
            print(p1,p2,G,t, pump_num)
            raise Exception('here')
        ef = (p2-p1)*1000*G/(U*I*rho)/np.sqrt(3)
        ef = 0 if ef < 0 or ef >= 1 else ef
        
        return ef / np.sqrt(3)
        
    else:
        return 0


pump_1_eff = df.apply(lambda x: eff_func(x.name, 11), axis=1)
pump_2_eff = df.apply(lambda x: eff_func(x.name, 12), axis=1)
pump_3_eff = df.apply(lambda x: eff_func(x.name, 13), axis=1)
pump_4_eff = df.apply(lambda x: eff_func(x.name, 14), axis=1)
pump_5_eff = df.apply(lambda x: eff_func(x.name, 15), axis=1)

pump_eff_data = {
    'pump_1_eff':pump_1_eff,
    'pump_2_eff':pump_2_eff,
    'pump_3_eff':pump_3_eff,
    'pump_4_eff':pump_4_eff,
    'pump_5_eff':pump_5_eff,
}



In [ ]:
fig = pump_eff_data['pump_1_eff'].plot(title="Показания ПЭН-I")
plt.ylabel("КПД насоса", fontsize=12)


<h1>Разделение на сегменты</h1>

In [ ]:
def divide_to_segments_by_eff(column, limiter):
    segm_start = None
    segm_end = None

    segments = []

    for ind,item in pump_eff_data[column].iteritems():
        if item>limiter:
            if segm_start is None:
                segm_start = ind
        else:
            if segm_start is not None and ind != segm_start:
                segm_end = ind

        if segm_start is not None and segm_end is not None:
            segments.append((segm_start-2,segm_end-3))
            segm_start, segm_end = None, None
    
    return segments

def divide_to_segments(column, limiter):
    segm_start = None
    segm_end = None

    segments = []

    for ind,item in df[column].iteritems():
        if item>limiter:
            if segm_start is None:
                segm_start = ind
        else:
            if segm_start is not None and ind != segm_start:
                segm_end = ind

        if segm_start is not None and segm_end is not None:
            segments.append((segm_start,segm_end))
            segm_start, segm_end = None, None
    
    return segments
        
# segments1 = divide_to_segments(pump_mass_cols[0], 350)
# segments2 = divide_to_segments(pump_mass_cols[1], 350)
# segments3 = divide_to_segments(pump_mass_cols[2], 440)
# segments4 = divide_to_segments(pump_mass_cols[3], 400)
# segments5 = divide_to_segments(pump_mass_cols[4], 400)

segments1 = divide_to_segments_by_eff('pump_1_eff', 0.1)
segments2 = divide_to_segments_by_eff('pump_2_eff', 0.1)
segments3 = divide_to_segments_by_eff('pump_3_eff', 0.1)
segments4 = divide_to_segments_by_eff('pump_4_eff', 0.1)
segments5 = divide_to_segments_by_eff('pump_5_eff', 0.1)


pump_mass_1_filtred = pd.concat([df[pump_mass_cols[0]][i[0]-2:i[1]-3] for i in segments1],  ignore_index=True)

filtred_data = {
    'pump_1_in': pd.concat([df[pump_in_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_in': pd.concat([df[pump_in_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_in': pd.concat([df[pump_in_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_in': pd.concat([df[pump_in_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_in': pd.concat([df[pump_in_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    'pump_1_out': pd.concat([df[pump_in_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_out': pd.concat([df[pump_in_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_out': pd.concat([df[pump_in_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_out': pd.concat([df[pump_in_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_out': pd.concat([df[pump_in_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    
    'pump_1_mass': pd.concat([df[pump_mass_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_mass': pd.concat([df[pump_mass_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_mass': pd.concat([df[pump_mass_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_mass': pd.concat([df[pump_mass_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_mass': pd.concat([df[pump_mass_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
 
    'pump_1_eff': pd.concat([pump_1_eff[i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_eff': pd.concat([pump_2_eff[i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_eff': pd.concat([pump_3_eff[i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_eff': pd.concat([pump_4_eff[i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_eff': pd.concat([pump_5_eff[i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    'pump_1_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_temp': pd.concat([df[pump_temp_col][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
    
    'pump_1_amper': pd.concat([df[pump_amper_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_amper': pd.concat([df[pump_amper_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_amper': pd.concat([df[pump_amper_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_amper': pd.concat([df[pump_amper_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_amper': pd.concat([df[pump_amper_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),

    'pump_1_revs': pd.concat([df[pump_revs_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
    'pump_2_revs': pd.concat([df[pump_revs_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
    'pump_3_revs': pd.concat([df[pump_revs_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
    'pump_4_revs': pd.concat([df[pump_revs_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
    'pump_5_revs': pd.concat([df[pump_revs_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
    
#     'pump_1_volt': pd.concat([df[pump_volt_cols[0]][i[0]:i[1]] for i in segments1],  ignore_index=True),
#     'pump_2_volt': pd.concat([df[pump_volt_cols[1]][i[0]:i[1]] for i in segments2],  ignore_index=True),
#     'pump_3_volt': pd.concat([df[pump_volt_cols[2]][i[0]:i[1]] for i in segments3],  ignore_index=True),
#     'pump_4_volt': pd.concat([df[pump_volt_cols[3]][i[0]:i[1]] for i in segments4],  ignore_index=True),
#     'pump_5_volt': pd.concat([df[pump_volt_cols[4]][i[0]:i[1]] for i in segments5],  ignore_index=True),
}
    
    
filtred_df = pd.DataFrame(filtred_data)

# for i in segments1:
#     print('--',pump1_eff[i[0]], pump1_eff[i[1]])
#     pump1_eff[i[0]:i[1]].plot()

# filtred_data['pump_2_eff'].plot(title='КПД ПЭН-II после фильтрации')

In [ ]:
filtred_data['pump_1_mass'].plot(title="Показания ПЭН-I")
plt.ylabel("Массовый расход, кг/c", fontsize=15)

rolled = pump_mass_1_filtred.rolling(50).mean()
plt.legend(loc='lower right')

# rolled.plot(linewidth=3)

<h1>Сглаживание</h1>

In [ ]:
def smooth(data, rolling=10, min_periods=1):
    f_max = data.max()
    f_norm = data/f_max
    return f_norm.rolling(rolling, min_periods).mean()

pump_mass_1_filtred
pump_mass_1_max = filtred_data['pump_1_mass'].max()

pump_mass_1_norm = filtred_data['pump_1_mass'] / pump_mass_1_max
pump_mass_1_norm.plot(title="Показания ПЭН-I", label="Нормализованный массовый расход")
# plt.ylabel("Массовый расход", fontsize=15)

rolled_mass_1 = pump_mass_1_norm.rolling(20, min_periods=1).mean()
rolled_mass_1.plot(label='Сглаженные данные', linewidth=3)
plt.ylabel("Массовый расход, кг/c", fontsize=12)
plt.legend(loc='lower right')

smoothed_data = {
    'pump_1_in':smooth(filtred_data['pump_1_in']),
    'pump_2_in':smooth(filtred_data['pump_2_in']),
    'pump_3_in':smooth(filtred_data['pump_3_in']),
    'pump_4_in':smooth(filtred_data['pump_4_in']),
    'pump_5_in':smooth(filtred_data['pump_5_in']),
    
    
    'pump_1_out':smooth(filtred_data['pump_1_out']),
    'pump_2_out':smooth(filtred_data['pump_2_out']),
    'pump_3_out':smooth(filtred_data['pump_3_out']),
    'pump_4_out':smooth(filtred_data['pump_4_out']),
    'pump_5_out':smooth(filtred_data['pump_5_out']),
    
    
    'pump_1_mass':smooth(filtred_data['pump_1_mass']),
    'pump_2_mass':smooth(filtred_data['pump_2_mass']),
    'pump_3_mass':smooth(filtred_data['pump_3_mass']),
    'pump_4_mass':smooth(filtred_data['pump_4_mass']),
    'pump_5_mass':smooth(filtred_data['pump_5_mass']),
    
    'pump_1_eff':smooth(filtred_data['pump_1_eff']),
    'pump_2_eff':smooth(filtred_data['pump_2_eff']),
    'pump_3_eff':smooth(filtred_data['pump_3_eff']),
    'pump_4_eff':smooth(filtred_data['pump_4_eff']),
    'pump_5_eff':smooth(filtred_data['pump_5_eff']),
    
    'pump_1_temp':smooth(filtred_data['pump_1_temp'], rolling=10),
    'pump_2_temp':smooth(filtred_data['pump_2_temp'], rolling=10),
    'pump_3_temp':smooth(filtred_data['pump_3_temp'], rolling=10),
    'pump_4_temp':smooth(filtred_data['pump_4_temp'], rolling=10),
    'pump_5_temp':smooth(filtred_data['pump_5_temp'], rolling=10),
    
    'pump_1_revs':smooth(filtred_data['pump_1_revs']),
    'pump_2_revs':smooth(filtred_data['pump_2_revs']),
    'pump_3_revs':smooth(filtred_data['pump_3_revs']),
    'pump_4_revs':smooth(filtred_data['pump_4_revs']),
    'pump_5_revs':smooth(filtred_data['pump_5_revs']),
    
    'pump_1_amper':smooth(filtred_data['pump_1_amper']),
    'pump_2_amper':smooth(filtred_data['pump_2_amper']),
    'pump_3_amper':smooth(filtred_data['pump_3_amper']),
    'pump_4_amper':smooth(filtred_data['pump_4_amper']),
    'pump_5_amper':smooth(filtred_data['pump_5_amper']),
}



In [ ]:
smoothed_data['pump_5_temp'].plot()


In [ ]:
(filtred_data['pump_1_in']/filtred_data['pump_1_in'].max()).plot(title='Показания ПЭН-I', label="Нормализованное давление на всасе")
smoothed_data['pump_1_in'].plot(linewidth=3, label="Сглаженные данные")
plt.ylabel("Давление на всасе", fontsize=12)
plt.legend(loc="lower right")


<h1>Центрирование</h1>

In [ ]:
# norm_pump_1_in = (filtred_data['pump_1_in']-filtred_data['pump_1_in'].mean())/filtred_data['pump_1_in']
# norm_pump_1_eff = (filtred_data['pump_1_eff']-filtred_data['pump_1_eff'].mean())/filtred_data['pump_1_eff']

# norm_pump_2_in = (filtred_data['pump_2_in']-filtred_data['pump_2_in'].mean())/filtred_data['pump_2_in']
# norm_pump_2_eff = (filtred_data['pump_2_eff']-filtred_data['pump_2_eff'].mean())/filtred_data['pump_2_eff']

# norm_pump_3_in = (filtred_data['pump_3_in']-filtred_data['pump_3_in'].mean())/filtred_data['pump_3_in']
# norm_pump_3_eff = (filtred_data['pump_3_eff']-filtred_data['pump_3_eff'].mean())/filtred_data['pump_3_eff']

In [ ]:
# corr1 = np.corrcoef(norm_pump_1_in.to_numpy(), norm_pump_1_eff.to_numpy())
# corr2 = np.corrcoef(norm_pump_2_in.to_numpy(), norm_pump_2_eff.to_numpy())
# corr3 = np.corrcoef(norm_pump_3_in.to_numpy(), norm_pump_3_eff.to_numpy())


# norm_pump_1_eff
# print(corr1[0][1], corr2[0][1], corr3[0][1])
# filtred_df['pump_2_in']
# df[pump_in_cols[1]].plot()

In [ ]:
norm_pump_1_in = (smoothed_data['pump_1_in']-smoothed_data['pump_1_in'].mean())/smoothed_data['pump_1_in']
norm_pump_1_eff = (smoothed_data['pump_1_eff']-smoothed_data['pump_1_eff'].mean())/smoothed_data['pump_1_eff']

norm_pump_2_in = (smoothed_data['pump_2_in']-smoothed_data['pump_2_in'].mean())/smoothed_data['pump_2_in']
norm_pump_2_eff = (smoothed_data['pump_2_eff']-smoothed_data['pump_2_eff'].mean())/smoothed_data['pump_2_eff']

norm_pump_3_in = (smoothed_data['pump_3_in']-smoothed_data['pump_3_in'].mean())/smoothed_data['pump_3_in']
norm_pump_3_eff = (smoothed_data['pump_3_eff']-smoothed_data['pump_3_eff'].mean())/smoothed_data['pump_3_eff']

centred_data = {
    'pump_1_in': (smoothed_data['pump_1_in']-smoothed_data['pump_1_in'].mean())/smoothed_data['pump_1_in'],
    'pump_2_in': (smoothed_data['pump_2_in']-smoothed_data['pump_2_in'].mean())/smoothed_data['pump_2_in'],
    'pump_3_in': (smoothed_data['pump_3_in']-smoothed_data['pump_3_in'].mean())/smoothed_data['pump_3_in'],
    'pump_4_in': (smoothed_data['pump_4_in']-smoothed_data['pump_4_in'].mean())/smoothed_data['pump_4_in'],
    'pump_5_in': (smoothed_data['pump_5_in']-smoothed_data['pump_5_in'].mean())/smoothed_data['pump_5_in'],

    'pump_1_out': (smoothed_data['pump_1_out']-smoothed_data['pump_1_out'].mean())/smoothed_data['pump_1_out'],
    'pump_2_out': (smoothed_data['pump_2_out']-smoothed_data['pump_2_out'].mean())/smoothed_data['pump_2_out'],
    'pump_3_out': (smoothed_data['pump_3_out']-smoothed_data['pump_3_out'].mean())/smoothed_data['pump_3_out'],
    'pump_4_out': (smoothed_data['pump_4_out']-smoothed_data['pump_4_out'].mean())/smoothed_data['pump_4_out'],
    'pump_5_out': (smoothed_data['pump_5_out']-smoothed_data['pump_5_out'].mean())/smoothed_data['pump_5_out'],

    'pump_1_eff': (smoothed_data['pump_1_eff']-smoothed_data['pump_1_eff'].mean())/smoothed_data['pump_1_eff'],
    'pump_2_eff': (smoothed_data['pump_2_eff']-smoothed_data['pump_2_eff'].mean())/smoothed_data['pump_2_eff'],
    'pump_3_eff': (smoothed_data['pump_3_eff']-smoothed_data['pump_3_eff'].mean())/smoothed_data['pump_3_eff'],
    'pump_4_eff': (smoothed_data['pump_4_eff']-smoothed_data['pump_4_eff'].mean())/smoothed_data['pump_4_eff'],
    'pump_5_eff': (smoothed_data['pump_5_eff']-smoothed_data['pump_5_eff'].mean())/smoothed_data['pump_5_eff'],
    
    'pump_1_temp': (smoothed_data['pump_1_temp']-smoothed_data['pump_1_temp'].mean())/smoothed_data['pump_1_temp'],
    'pump_2_temp': (smoothed_data['pump_2_temp']-smoothed_data['pump_2_temp'].mean())/smoothed_data['pump_2_temp'],
    'pump_3_temp': (smoothed_data['pump_3_temp']-smoothed_data['pump_3_temp'].mean())/smoothed_data['pump_3_temp'],
    'pump_4_temp': (smoothed_data['pump_4_temp']-smoothed_data['pump_4_temp'].mean())/smoothed_data['pump_4_temp'],
    'pump_5_temp': (smoothed_data['pump_5_temp']-smoothed_data['pump_5_temp'].mean())/smoothed_data['pump_5_temp'],

    'pump_1_revs': (smoothed_data['pump_1_revs']-smoothed_data['pump_1_revs'].mean())/smoothed_data['pump_1_revs'],
    'pump_2_revs': (smoothed_data['pump_2_revs']-smoothed_data['pump_2_revs'].mean())/smoothed_data['pump_2_revs'],
    'pump_3_revs': (smoothed_data['pump_3_revs']-smoothed_data['pump_3_revs'].mean())/smoothed_data['pump_3_revs'],
    'pump_4_revs': (smoothed_data['pump_4_revs']-smoothed_data['pump_4_revs'].mean())/smoothed_data['pump_4_revs'],
    'pump_5_revs': (smoothed_data['pump_5_revs']-smoothed_data['pump_5_revs'].mean())/smoothed_data['pump_5_revs'],

    'pump_1_amper': (smoothed_data['pump_1_amper']-smoothed_data['pump_1_amper'].mean())/smoothed_data['pump_1_amper'],
    'pump_2_amper': (smoothed_data['pump_2_amper']-smoothed_data['pump_2_amper'].mean())/smoothed_data['pump_2_amper'],
    'pump_3_amper': (smoothed_data['pump_3_amper']-smoothed_data['pump_3_amper'].mean())/smoothed_data['pump_3_amper'],
    'pump_4_amper': (smoothed_data['pump_4_amper']-smoothed_data['pump_4_amper'].mean())/smoothed_data['pump_4_amper'],
    'pump_5_amper': (smoothed_data['pump_5_amper']-smoothed_data['pump_5_amper'].mean())/smoothed_data['pump_5_amper'],
}

corr_coefs = [
    # p_in -- p_eff
    np.corrcoef(centred_data['pump_1_in'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_in'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_in'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_in'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_in'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),

    # p_out -- p_eff
    np.corrcoef(centred_data['pump_1_out'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_out'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_out'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_out'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_out'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),

    
    # p_temp -- p_eff
    np.corrcoef(centred_data['pump_1_temp'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_temp'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_temp'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_temp'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_temp'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),    
    
    # p_I -- p_eff
    np.corrcoef(centred_data['pump_1_amper'].to_numpy(), centred_data['pump_1_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_2_amper'].to_numpy(), centred_data['pump_2_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_3_amper'].to_numpy(), centred_data['pump_3_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_4_amper'].to_numpy(), centred_data['pump_4_eff'].to_numpy()),
    np.corrcoef(centred_data['pump_5_amper'].to_numpy(), centred_data['pump_5_eff'].to_numpy()),    
]


for ind, coef in enumerate(corr_coefs):
    if ind % 5 == 0:
        print("---------------")
    print(coef[0][1])


In [ ]:
# # df.rename(columns={
# #     pump_in_cols[0]: "pump_1_in",
# #     pump_in_cols[1]: "pump_2_in",
# #     pump_in_cols[2]: "pump_3_in",
# #     pump_in_cols[3]: "pump_4_in",
# #     pump_in_cols[4]: "pump_5_in",
    
# #     pump_out_cols[0]: "pump_1_out",
# #     pump_out_cols[1]: "pump_2_out",
# #     pump_out_cols[2]: "pump_3_out",
# #     pump_out_cols[3]: "pump_4_out",
# #     pump_out_cols[4]: "pump_5_out",
    
# #     pump_mass_cols[0]: "pump_1_mass",
# #     pump_mass_cols[1]: "pump_2_mass",
# #     pump_mass_cols[2]: "pump_3_mass",
# #     pump_mass_cols[3]: "pump_4_mass",
# #     pump_mass_cols[4]: "pump_5_mass",
    
# #     pump_temp_col: "temp",
    
# #     pump_amper_cols[0]: "pump_1_amper",
# #     pump_amper_cols[1]: "pump_2_amper",
# #     pump_amper_cols[2]: "pump_3_amper",
# #     pump_amper_cols[3]: "pump_4_amper",
# #     pump_amper_cols[4]: "pump_5_amper",
    
# #     pump_revs_cols[0]: "pump_1_revs",
# #     pump_revs_cols[1]: "pump_2_revs",
# #     pump_revs_cols[2]: "pump_3_revs",
# #     pump_revs_cols[3]: "pump_4_revs",
# #     pump_revs_cols[4]: "pump_5_revs",
    
# #     pump_volt_cols[0]: "pump_1_volt",
# #     pump_volt_cols[1]: "pump_2_volt",
# #     pump_volt_cols[2]: "pump_3_volt",
# #     pump_volt_cols[3]: "pump_4_volt",
    
# #     pump_turbine_col: "pump_turbine",
# #     pump_reactor_col: "pump_reactor",
# # }, inplace = True)

# df.rename(columns={
#     pump_in_cols[0]: "ПЭН-1 Pнап",
#     pump_in_cols[1]: "ПЭН-2 Pнап",
#     pump_in_cols[2]: "ПЭН-3 Pнап",
#     pump_in_cols[3]: "ПЭН-4 Pнап",
#     pump_in_cols[4]: "ПЭН-5 Pнап",
    
#     pump_out_cols[0]: "ПЭН-1 Pвсас",
#     pump_out_cols[1]: "ПЭН-2 Pвсас",
#     pump_out_cols[2]: "ПЭН-3 Pвсас",
#     pump_out_cols[3]: "ПЭН-4 Pвсас",
#     pump_out_cols[4]: "ПЭН-5 Pвсас",
    
#     pump_mass_cols[0]: "ПЭН-1 Расход",
#     pump_mass_cols[1]: "ПЭН-2 Расход",
#     pump_mass_cols[2]: "ПЭН-3 Расход",
#     pump_mass_cols[3]: "ПЭН-4 Расход",
#     pump_mass_cols[4]: "ПЭН-5 Расход",
    
#     pump_temp_col: "Tпит.в",
    
#     pump_amper_cols[0]: "ПЭН-1 Сила тока",
#     pump_amper_cols[1]: "ПЭН-2 Сила тока",
#     pump_amper_cols[2]: "ПЭН-3 Сила тока",
#     pump_amper_cols[3]: "ПЭН-4 Сила тока",
#     pump_amper_cols[4]: "ПЭН-5 Сила тока",
    
#     pump_revs_cols[0]: "ПЭН-1 Част.вр.эл.дв.",
#     pump_revs_cols[1]: "ПЭН-2 Част.вр.эл.дв.",
#     pump_revs_cols[2]: "ПЭН-3 Част.вр.эл.дв.",
#     pump_revs_cols[3]: "ПЭН-4 Част.вр.эл.дв.",
#     pump_revs_cols[4]: "ПЭН-5 Част.вр.эл.дв.",
    
#     pump_volt_cols[0]: "Напряж. секц.BBA",
#     pump_volt_cols[1]: "Напряж. секц.BBB",
#     pump_volt_cols[2]: "Напряж. секц.BBC",
#     pump_volt_cols[3]: "Напряж. секц.BBD",
    
#     pump_turbine_col: "Мощность турб",
#     pump_reactor_col: "Мощность реакт",
# }, inplace = True)


# df.corr()